In [1]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
from keras.models import model_from_yaml
from keras.models import model_from_json
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [2]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds

In [3]:
df = pd.read_csv("../Datasets/Cuvinte-Eliminate/train-punct-stop-1000.csv")
df=df.dropna()
df=df.sample(3000000)
#df=df.drop(["news_title","url","reddit_title"],axis=1)

In [4]:
df.head()

,sentiment,text
778585,2,another superior effort el arrangements vocals...
2254431,0,though light product easy store fold gets soon...
3018524,0,not enough
405382,2,pattern design identical love use sons right n...
722781,2,ive listening dire favourite album favourite s...


In [5]:
df.count()

sentiment    1000000
text         1000000
dtype: int64

In [6]:
data_clean = df.text

In [7]:
data_labels = df.sentiment.values

In [8]:
data_clean

778585     another superior effort el arrangements vocals...
2254431    though light product easy store fold gets soon...
3018524                                          not enough 
405382     pattern design identical love use sons right n...
722781     ive listening dire favourite album favourite s...
                                 ...                        
2754325    guys come reviews talk yummy bar checked nutri...
1077830    nice phone no reception problems bluetooth wor...
1036139    remember movie came not interest time remarkab...
678305     wasnt worst season no fate would fall final 7 ...
2775991    sorry put one big disappointment loved eddie c...
Name: text, Length: 1000000, dtype: object

In [9]:
data_labels




array([2, 0, 0, ..., 2, 2, 0])

In [10]:
data_labels_good=[]
for i in data_labels:
    if i==2:
        data_labels_good.append(1)
    else:
        data_labels_good.append(0)

data_labels_good=np.array(data_labels_good)

In [11]:
data_labels_good

array([1, 0, 0, ..., 1, 1, 0])

In [12]:
set(data_labels_good)

{0, 1}

In [13]:
import pickle
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [14]:
max_phrase_len=200

In [15]:
X_train = tokenizer.texts_to_sequences(data_clean)
X_train = pad_sequences(X_train, maxlen = max_phrase_len)

In [16]:
len(X_train[1])

200

In [17]:
class DCNN(tf.keras.Model):

    def __init__(self,
                 vocab_size,
                 emb_dim=128,
                 nb_filters=50,
                 FFN_units=512,
                 nb_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="dcnn"):
        super(DCNN, self).__init__(name=name)
        self.embedding = layers.Embedding(vocab_size,
                                          emb_dim)
        self.bigram = layers.Conv1D(filters=nb_filters,
                                    kernel_size=2,
                                    padding="valid",
                                    activation="relu")
        self.pool_1 = layers.GlobalMaxPool1D()
        self.trigram = layers.Conv1D(filters=nb_filters,
                                     kernel_size=3,
                                     padding="valid",
                                     activation="relu")
        self.pool_2 = layers.GlobalMaxPool1D()
        self.fourgram = layers.Conv1D(filters=nb_filters,
                                      kernel_size=4,
                                      padding="valid",
                                      activation="relu")
        
        self.pool_3 = layers.GlobalMaxPool1D()
        self.dense_1 = layers.Dense(units=FFN_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if nb_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=nb_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        x = self.embedding(inputs)
        x_1 = self.bigram(x)
        x_1 = self.pool_1(x_1)
        x_2 = self.trigram(x)
        x_2 = self.pool_2(x_2)
        x_3 = self.fourgram(x)
        x_3 = self.pool_3(x_3)

        merged = tf.concat([x_1, x_2, x_3], axis=-1) # (batch_size, 3 * nb_filters)
        merged = self.dense_1(merged)
        merged = self.dropout(merged, training)
        output = self.last_dense(merged)

        return output

In [18]:
#VOCAB_SIZE = 16384
VOCAB_SIZE = 8192
EMB_DIM = 200
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = len(set(data_labels_good))

DROPOUT_RATE = 0.5

BATCH_SIZE = 265
NB_EPOCHS = 3

In [19]:
Dcnn = DCNN(vocab_size=VOCAB_SIZE,
            emb_dim=EMB_DIM,
            nb_filters=NB_FILTERS,
            FFN_units=FFN_UNITS,
            nb_classes=NB_CLASSES,
            dropout_rate=DROPOUT_RATE)

2022-02-06 14:51:35.368274: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-06 14:51:35.380029: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-06 14:51:35.380729: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-06 14:51:35.382586: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [20]:
if NB_CLASSES == 2:
    Dcnn.compile(loss="binary_crossentropy",
                 optimizer="adam",
                 metrics=["accuracy"])
else:
    Dcnn.compile(loss="sparse_categorical_crossentropy",
                 optimizer="adam",
                 metrics=["sparse_categorical_accuracy"])

In [21]:
Dcnn.fit(X_train,
         data_labels_good,
         batch_size=BATCH_SIZE,
         validation_split=0.2,
         shuffle=True,
         verbose=1,
         epochs=NB_EPOCHS)

2022-02-06 14:51:36.816061: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/3


2022-02-06 14:51:38.136169: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8204


3019/3019 [==============================] - 271s 89ms/step - loss: 0.2719 - accuracy: 0.8864 - val_loss: 0.2447 - val_accuracy: 0.8984
Epoch 2/3
3019/3019 [==============================] - 279s 92ms/step - loss: 0.2081 - accuracy: 0.9178 - val_loss: 0.2397 - val_accuracy: 0.9026
Epoch 3/3
3019/3019 [==============================] - 283s 94ms/step - loss: 0.1598 - accuracy: 0.9389 - val_loss: 0.2564 - val_accuracy: 0.9011


In [22]:
Dcnn.summary()

Model: "dcnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  1638400   
_________________________________________________________________
conv1d (Conv1D)              multiple                  40100     
_________________________________________________________________
global_max_pooling1d (Global multiple                  0         
_________________________________________________________________
conv1d_1 (Conv1D)            multiple                  60100     
_________________________________________________________________
global_max_pooling1d_1 (Glob multiple                  0         
_________________________________________________________________
conv1d_2 (Conv1D)            multiple                  80100     
_________________________________________________________________
global_max_pooling1d_2 (Glob multiple                  0      

In [25]:
Dcnn.save("./Model90/")

INFO:tensorflow:Assets written to: ./Model90/assets


In [24]:
model_yaml = Dcnn.to_json()
with open("model.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
model.save_weights("model.h5")

NotImplementedError: 